In [8]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

print('torch', torch.__version__)
print('cuda', torch.cuda.is_available())


torch 1.3.1
cuda True


In [2]:
batch_size = 64
momentum = 0.5
test_batch_size = 1000
lr = 0.01
log_interval = 10
epochs = 10

use_cuda = torch.cuda.is_available()
seed = 3123
torch.manual_seed(seed)
if use_cuda:
    torch.cuda.manual_seed(seed)

In [13]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net()
if use_cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [14]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303909
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.314535
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.296031
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.272993
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.249559
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.230603
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.205716
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.249150
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.180456
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.067198
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.997476
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.956994
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.798316
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.578809
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.697945
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.391784
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.452057
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.101846
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.051645
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.129787
Train Epoch: 1 [12800/60000 (

/home/ctarrington/anaconda3/envs/for-pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.1909, Accuracy: 9434/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.462906
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.413128
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.612110
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.576819
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.607273
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.468020
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.519264
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.310091
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.467509
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.815569
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.325485
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.474923
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.683145
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.520883
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.319498
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.718691
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.399938
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.372728
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.448594
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.618553
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.410553
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.160525
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.201330
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.276801
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.388614
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.233959
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.356934
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.341853
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.279405
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.284549
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.240026
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.119370
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.640543
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.301601
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.359045
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.408533
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.272667
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.262116
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.303104


Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.100667
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.193595
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.153116
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.127922
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.377832
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.107748
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.363138
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.116652
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.318377
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.154224
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.102220
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.233369
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.148806
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.280556
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.126335
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.183264
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.152405
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.180640
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.270679
Train Epoch: 5 [40960/60000 (68%)]	Loss: 0.258115


Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.214236
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.213386
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.068349
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.064717
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.185362
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.278383
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.183645
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.143691
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.271610
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.197972
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.294658
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.208270
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.044680
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.177494
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.307907
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.232592
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.162896
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.229723
Train Epoch: 7 [24960/60000 (42%)]	Loss: 0.067880
Train Epoch: 7 [25600/60000 (43%)]	Loss: 0.283899


Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.063285

Test set: Average loss: 0.0544, Accuracy: 9814/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.225174
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.275731
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.175722
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.246077
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.172272
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.259864
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.086337
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.208748
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.342250
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.156918
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.081014
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.215732
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.197793
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.293867
Train Epoch: 9 [8960/60000 (15%)]	Loss: 0.074018
Train Epoch: 9 [9600/60000 (16%)]	Loss: 0.209599
Train Epoch: 9 [10240/60000 (17%)]	Loss: 0.176739
Train Epoch: 9 [10880/60000 (18%)]	Loss: 0.197286
Train Epoch: 9 [11

Train Epoch: 10 [44160/60000 (74%)]	Loss: 0.151053
Train Epoch: 10 [44800/60000 (75%)]	Loss: 0.130625
Train Epoch: 10 [45440/60000 (76%)]	Loss: 0.123497
Train Epoch: 10 [46080/60000 (77%)]	Loss: 0.083894
Train Epoch: 10 [46720/60000 (78%)]	Loss: 0.089352
Train Epoch: 10 [47360/60000 (79%)]	Loss: 0.057763
Train Epoch: 10 [48000/60000 (80%)]	Loss: 0.145854
Train Epoch: 10 [48640/60000 (81%)]	Loss: 0.362942
Train Epoch: 10 [49280/60000 (82%)]	Loss: 0.288404
Train Epoch: 10 [49920/60000 (83%)]	Loss: 0.193108
Train Epoch: 10 [50560/60000 (84%)]	Loss: 0.187923
Train Epoch: 10 [51200/60000 (85%)]	Loss: 0.255062
Train Epoch: 10 [51840/60000 (86%)]	Loss: 0.142383
Train Epoch: 10 [52480/60000 (87%)]	Loss: 0.122364
Train Epoch: 10 [53120/60000 (88%)]	Loss: 0.275331
Train Epoch: 10 [53760/60000 (90%)]	Loss: 0.113078
Train Epoch: 10 [54400/60000 (91%)]	Loss: 0.373991
Train Epoch: 10 [55040/60000 (92%)]	Loss: 0.256147
Train Epoch: 10 [55680/60000 (93%)]	Loss: 0.164383
Train Epoch: 10 [56320/60000 (9